# Run the mass balance calibration
From the [OGGM documentation](http://docs.oggm.org/en/latest/run_examples/run_mb_calibration.html):
> Sometimes you will need to do the mass-balance calibration yourself. For example if you use alternate climate data, or change the parameters of the model. Here we show how to run the calibration for all available reference glaciers, but you can also doit for any regional subset of course.
The output of this script is the `ref_tstars.csv` file, which is found in the working directory. The `ref_tstars.csv` file can then be used for further runs, simply by copying it in the corresponding working directory before the run.

Since I use a different (mass balance) model, I'll have to do the calibration on my own. The resulting `ref_tstars.csv` file will some day be included in the OGGM sample data, once my model goes public.

In [1]:
# Python imports
import json
import os

# Libs
import numpy as np

# Locals
import oggm
from oggm import cfg, utils, tasks, workflow
from oggm.workflow import execute_entity_task
from oggm.core.massbalance import (ConstantMassBalance, PastMassBalance,
                                   MultipleFlowlineMassBalance)

A different outline (i.e. different RGI version) alters the outcome of the mass balance model. Same goes for the used baseline climate, most obviously. This is why the used data is specified hereafter, so that the reference list can be approprated to the right data. Additionally, the local paths are defined 

In [2]:
# RGI Version
rgi_version = '61'

# Since I'm in the Alps, I'll use HistAlp as baseline climate
baseline = 'HISTALP'

# Initialize OGGM and set up the run parameters
cfg.initialize(logging_level='WORKFLOW')

# Local paths (where to write the OGGM run output)
dirname = 'OGGM_ref_mb_{}_RGIV{}_OGGM{}'.format(baseline, rgi_version,
                                                oggm.__version__)
WORKING_DIR = os.path.join(os.path.abspath('.'), dirname)
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR
# print to console
print(WORKING_DIR)

2019-02-27 15:11:24: oggm.cfg: Using configuration file: /Users/oberrauch/oggm-fork/oggm/params.cfg


/Users/oberrauch/work/master/notebooks/OGGM_ref_mb_HISTALP_RGIV61_OGGM1.0.3+149.g1dd6f37.dirty


The following code block alters certain parameters from the default config file.

In [3]:
# We are running the calibration ourselves
cfg.PARAMS['run_mb_calibration'] = True

# We are using which baseline data?
cfg.PARAMS['baseline_climate'] = baseline

# No need for intersects since this has an effect on the inversion only
cfg.PARAMS['use_intersects'] = False

# Use multiprocessing?
cfg.PARAMS['use_multiprocessing'] = True

# Set to True for operational runs
cfg.PARAMS['continue_on_error'] = False

# Other params: see https://oggm.org/2018/08/10/histalp-parameters/
# TODO: do I have to calibrate those parameters again, since I'm using
# a different mass balance model?!
#cfg.PARAMS['baseline_y0'] = 1850
cfg.PARAMS['prcp_scaling_factor'] = 1.75
cfg.PARAMS['temp_melt'] = -1.75

The next step is to get all the reference glaciers, i.e. glaciers with mass balance measurements.

In [4]:
# Get the reference glacier ids (they are different for each RGI version)
rgi_dir = utils.get_rgi_dir(version=rgi_version)
df, _ = utils.get_wgms_files()
rids = df['RGI{}0_ID'.format(rgi_version[0])]


# We can't do Antarctica
rids = [rid for rid in rids if not ('-19.' in rid)]

# For HISTALP only RGI reg 11
if baseline == 'HISTALP':
    rids = [rid for rid in rids if '-11.' in rid]

# Make a new dataframe with those (this takes a while)
print('Reading the RGI shapefiles...')
rgidf = utils.get_rgi_glacier_entities(rids, version=rgi_version)
print('For RGIV{} we have {} candidate reference '
      'glaciers.'.format(rgi_version, len(rgidf)))

Reading the RGI shapefiles...
For RGIV61 we have 59 candidate reference glaciers.


In [5]:
# save those reference glaciers in a seperate DataFrame
rgidf.to_csv('/Users/oberrauch/work/master/data/rgi_ref_glaciers.csv')

In [6]:
# initialize the glacier regions
gdirs = workflow.init_glacier_regions(rgidf)

2019-02-27 15:11:28: oggm.workflow: Execute entity task define_glacier_region on 59 glaciers
2019-02-27 15:11:28: oggm.workflow: Multiprocessing: using all available processors (N=4)


In [7]:
# We need to know which period we have data for
print('Process the climate data...')
cfg.PARAMS['continue_on_error'] = True  # Some glaciers are not in Alps
execute_entity_task(tasks.process_histalp_data, gdirs, print_log=False)
cfg.PARAMS['continue_on_error'] = False

2019-02-27 15:11:29: oggm.workflow: Execute entity task process_histalp_data on 59 glaciers


Process the climate data...


2019-02-27 15:11:30: oggm.workflow: Multiprocessing: using all available processors (N=4)


In [10]:
# Save to file
rgidf.to_file(os.path.join(WORKING_DIR, 'mb_ref_glaciers.shp'))
print('For RGIV{} and {} we have {} reference glaciers.'.format(rgi_version,
                                                                baseline,
                                                                len(rgidf)))

For RGIV61 and HISTALP we have 56 reference glaciers.


In [11]:
# Sort for more efficient parallel computing
rgidf = rgidf.sort_values('Area', ascending=False)

# Go - initialize glacier directories
gdirs = workflow.init_glacier_regions(rgidf)

In [12]:
# Prepro tasks
task_list = [
    tasks.glacier_masks,
    tasks.compute_centerlines,
    tasks.initialize_flowlines,
    tasks.catchment_area,
    tasks.catchment_intersections,
    tasks.catchment_width_geom,
    tasks.catchment_width_correction,
]
for task in task_list:
    execute_entity_task(task, gdirs)

2019-02-27 15:11:46: oggm.workflow: Execute entity task glacier_masks on 56 glaciers
2019-02-27 15:11:46: oggm.workflow: Multiprocessing: using all available processors (N=4)
2019-02-27 15:11:50: oggm.workflow: Execute entity task compute_centerlines on 56 glaciers
2019-02-27 15:11:53: oggm.workflow: Execute entity task initialize_flowlines on 56 glaciers
2019-02-27 15:11:56: oggm.workflow: Execute entity task catchment_area on 56 glaciers
2019-02-27 15:12:25: oggm.workflow: Execute entity task catchment_intersections on 56 glaciers
2019-02-27 15:12:28: oggm.workflow: Execute entity task catchment_width_geom on 56 glaciers
2019-02-27 15:12:47: oggm.workflow: Execute entity task catchment_width_correction on 56 glaciers


In [ ]:
# run climate tasks
vascaling.compute_ref_t_stars(gdirs)
execute_entity_task(vascaling.local_t_star, gdirs)